# tests tab-NTV

In [1]:
import sys
sys.path

['C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Tests',
 'C:\\Users\\phili\\github\\NTV\\json_ntv',
 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\observation',
 'C:\\Users\\phili\\github\\NTV',
 'C:\\Users\\phili\\github\\Environmental-Sensing\\python',
 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Tests',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\python310.zip',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\DLLs',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309',
 '',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages\\win32',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\phili\\anaconda3\\envs\\observ_v10_20230309\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\phili\\anaconda3\\envs\\obser

In [2]:
from json_ntv import NtvSingle, NtvList, NtvSet, Ntv, NtvConnector
from json_ntv import NtvType, Namespace
from datetime import datetime, date, time


In [3]:
def periodic_coef(lis):
    if len(lis) < 2:
        return 0
    coef = 0
    period = max(lis) + 1
    for i in range(1,len(lis)):
        coef = i
        if lis[i-1] != lis[i]:
            break
    periodic_lis = [ (ikey % (coef * period)) // coef for ikey in range(len(lis))]
    if lis == periodic_lis:
        return coef
    return 0


In [4]:
from observation import Iindex
def decode_ntv_field(field, encode_format='json'):
    '''return tuple : name, type, codec, parent, keys'''
    ntv = Ntv.obj(field)
    type_ntv = ntv.ntv_type.long_name if ntv.ntv_type else None
    if isinstance(ntv, NtvSingle):
        return (ntv.ntv_name, type_ntv, [ntv.to_obj(simpleval=True, encode_format=encode_format)], 
                None, None)
    if len(ntv) == 0 or len(ntv) > 3:
        return (ntv.ntv_name, type_ntv, ntv.to_obj(simpleval=True, encode_format=encode_format), 
                None, None)
    codec_ntv = ntv[0]
    type_codec_ntv = codec_ntv.ntv_type.long_name if codec_ntv.ntv_type else None
    if len(ntv) == 3 and isinstance(ntv[1], NtvSingle) and isinstance(ntv[1].ntv_value, (int, str)) and \
      not isinstance(ntv[2], NtvSingle) and isinstance(ntv[2][0].ntv_value, int):
        return (ntv.ntv_name, type_codec_ntv, codec_ntv.to_obj(simpleval=True, encode_format=encode_format), 
                ntv[1].ntv_value, ntv[2].to_obj())
    if len(ntv) == 2 and isinstance(ntv[1], NtvSingle) and isinstance(ntv[1].ntv_value, (int, str)):
        return (ntv.ntv_name, type_codec_ntv, codec_ntv.to_obj(simpleval=True, encode_format=encode_format), 
                ntv[1].ntv_value, None) 
    if len(ntv) == 2 and not isinstance(ntv[1], NtvSingle) and isinstance(ntv[1][0].ntv_value, int):
        return (ntv.ntv_name, type_codec_ntv, codec_ntv.to_obj(simpleval=True, encode_format=encode_format), 
                None, ntv[1].to_obj())
    return (ntv.ntv_name, type_ntv, ntv.to_obj(simpleval=True, encode_format=encode_format), 
                None, None)

def from_ntv_iindex(ntv_value):
    name, typ, codec, parent, keys = decode_ntv_field(ntv_value, encode_format='json')
    if parent or (keys and len(keys) == 1):
        return None
    return Iindex(codec=codec, name=name, keys=keys, typevalue=None, reindex=True)

def to_ntv_iindex(self, modecodec='optimize', def_type=None, keys=None, parent=None):
    leng = len(self)
    if len(self.codec) == 1:
        return NtvSingle(self.codec[0], self.name)
    name = None if self.name == '$default' else self.name       
    if len(self.codec) == leng or modecodec == 'full':
        return NtvList(self.values, name)
    if modecodec == 'default':
        return NtvList([NtvList(self.codec, ntv_type=def_type), NtvList(self.keys, ntv_type='json')], name, ntv_type='json')
    if modecodec == 'optimize':
        ntv_value = [NtvList(self.codec, ntv_type=def_type)]
        if parent:
            ntv_value.append(NtvSingle(parent, ntv_type='json'))
        if keys:
            ntv_value.append(NtvList(keys, ntv_type='json'))    
        else:
            ntv_value.append(NtvList(self.keys, ntv_type='json'))
        return NtvList(ntv_value, name, ntv_type='json')                
    
idx_list = [ Iindex([1], 'unique'),
             Iindex([1,2,3], 'list'),
             Iindex([date(2000, 1, 1), date(2000, 2, 1)], 'list_date', [0,0,1]),
             Iindex([1,1,2,2,4,4,0,0], 'periodic')]

for idx in idx_list:
    print(to_ntv_iindex(idx, modecodec='full'))
    print(to_ntv_iindex(idx, modecodec='default'))
    print(to_ntv_iindex(idx, modecodec='optimize'), '\n')

print(to_ntv_iindex(idx, parent=1, keys=[0,1,0,1,1]))

    

{"unique": 1}
{"unique": 1}
{"unique": 1} 

{"list": [1, 2, 3]}
{"list": [1, 2, 3]}
{"list": [1, 2, 3]} 

{"list_date::date": ["2000-01-01", "2000-01-01", "2000-02-01"]}
{"list_date": [{"::date": ["2000-01-01", "2000-02-01"]}, [0, 0, 1]]}
{"list_date": [{"::date": ["2000-01-01", "2000-02-01"]}, [0, 0, 1]]} 

{"periodic": [1, 1, 2, 2, 4, 4, 0, 0]}
{"periodic": [1, 1, 2, 2, 4, 4, 0, 0]}
{"periodic": [1, 1, 2, 2, 4, 4, 0, 0]} 

{"periodic": [1, 1, 2, 2, 4, 4, 0, 0]}


In [5]:
fields = [{'full_dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21']},
          ['1964-01-01', '1985-02-05', '2022-01-21'],
          {'full_coord::point':    [[1,2], [3,4], [5,6]]},
          {'full_simple': [1,2,3,4]},
          {'complete_test': [['a', 'b'], [0, 0, 1, 0]]},
          {'complete_test': [['a', 'b'], [0, 0, 1, 0]]},
          {"complete_date": [{"::date": ["2000-01-01", "2000-02-01"]}, [0, 0, 1]]},
          {'implicit_test': [['a', 'b'], 'parent']},
          {'relative_test': [{'::string': ['a', 'b']}, 1, [0, 1, 1]]},
          [{'::string': ['a', 'b']}, 1, [0, 1, 1]],
          {'primary_test': [['a', 'b'], [2]]},
          [['a', 'b'], [2]],
          {'unic_test': 'valunic' },
          'valunic',
          {'primary': [['oui', 'fin 2022'], [1]]},
          [['oui', 'fin 2022'], [1]]]

field_old = [{'test': 'datvalue'}, ['a', 'b'], [1, [0, 0, 1, 0]]]

for field in fields:
    #print(decode_ntv_field(field, 'json'))
    idx = from_ntv_iindex(field)
    #print(idx, type(idx.values[0]))
    if idx:
        for mode in ['full', 'default', 'optimize']:
            print(to_ntv_iindex(idx, mode))
            print(idx == from_ntv_iindex(to_ntv_iindex(idx, mode)))

{"full_dates::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00"]}
True
{"full_dates::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00"]}
True
{"full_dates::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00"]}
True
{"::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00"]}
True
{"::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00"]}
True
{"::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00"]}
True
{"full_coord": [[1, 2], [3, 4], [5, 6]]}
True
{"full_coord": [[1, 2], [3, 4], [5, 6]]}
True
{"full_coord": [[1, 2], [3, 4], [5, 6]]}
True
{"full_simple": [1, 2, 3, 4]}
True
{"full_simple": [1, 2, 3, 4]}
True
{"full_simple": [1, 2, 3, 4]}
True
{"complete_test": ["a", "a", "b", "a"]}
True
{"complete_test": [["a", "b"

In [6]:
from observation import Ilist

def from_ntv_ilist(ntv_value, reindex=False):
    ntv = Ntv.obj(ntv_value)
    leng = max([len(ntvi) for ntvi in ntv.ntv_value])
    # decode: name, type, codec, parent, keys
    lidx = [list(decode_ntv_field(ntvf)) for ntvf in ntv]
    for ind in range(len(lidx)):
        if lidx[ind][0] == '':
            lidx[ind][0] = 'i'+str(ind)
        _init_keys(ind, lidx, leng)
    #print(lidx)
    lindex = [Iindex(idx[2], idx[0], idx[4], None, # idx[1] pour le type,
                 reindex=reindex) for idx in lidx]
    #print('\nlindex : ', lindex)
    return Ilist(lindex, reindex=False)


@staticmethod
def _init_keys(ind, lidx, leng):
    ''' initialization of explicit keys data in lidx object'''
    # name: 0, type: 1, codec: 2, parent: 3, keys: 4
    keys = lidx[ind][4]
    if keys is None and lidx[ind][3] is None:  # full or unique
        if len(lidx[ind][2]) == 1: # unique
            lidx[ind][4] = [0] * leng
        else:    # full
            lidx[ind][4] = list(range(leng))
        return
    if keys and len(keys) > 1 and lidx[ind][3] is None:  #complete
        return
    if keys and len(keys) == 1:  #primary
        coef = keys[0]
        lidx[ind][4] = [ (ikey % (coef * len(lidx[ind][2]))) // coef for ikey in range(leng)]
        return  
    parent = lidx[ind][3]
    if parent is None:
        raise IlistError('keys not referenced')          
    if not lidx[parent][4] or len(lidx[parent][4]) != leng:
        _init_keys(parent, lidx, leng)
    if not keys and len(lidx[ind][2]) == len(lidx[parent][2]):    # implicit
        lidx[ind][4] = lidx[parent][4]
        return
    lidx[ind][4] = Iindex.keysfromderkeys(lidx[parent][4], keys)  # relative
    return


In [7]:
def to_ntv_ilist(self, modecodec='optimize', def_type='json', option_name=False):
    if modecodec != 'optimize':
        lis = [to_ntv_iindex(index, modecodec=modecodec) for index in self.lindex]
    else:
        lis = []
        indexinfos = self.indexinfos()
        indexname = [option_name or name != 'i' + str(i)
                     for i, name in enumerate(self.lname)]
        for idx, iname, inf in zip(self.lindex, indexname, indexinfos):
            coef = periodic_coef(idx.keys)
            if inf['cat'] == 'unique':
                lis.append(to_ntv_iindex(idx))
            elif coef:
                lis.append(to_ntv_iindex(idx, keys=[coef]))
            elif inf['cat'] == 'coupled':
                idx_coup = idx.setkeys(self.lindex[inf['parent']].keys, inplace=False)
                lis.append(to_ntv_iindex(idx_coup, parent=inf['parent']))
            elif inf['parent'] == -1:  # cat='variable' or 'secondary'
                if idx.keys == list(range(len(self))):
                    lis.append(to_ntv_iindex(idx, modecodec='full'))
                else:
                    lis.append(to_ntv_iindex(idx, modecodec='default'))                
            else:  # derived
                if len(self.lindex[inf['parent']].codec) == len(self):
                    lis.append(to_ntv_iindex(idx, modecodec='default'))                
                elif idx.iskeysfromderkeys(self.lindex[inf['parent']]):
                    lis.append(to_ntv_iindex(idx, parent=inf['parent']))
                else: # periodic derived
                    keys = idx.derkeys(self.lindex[inf['parent']])
                    lis.append(to_ntv_iindex(idx, keys=keys, parent=inf['parent']))            
    return NtvList(lis, self.name, ntv_type=def_type)

In [21]:
tab_data   = {'value':           [10, 20, 30, 40],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21', '2022-01-22'],
              'value32::int32':  [10, 20, 30, 40],
              'coord::point':    [[1,2], [3,4], [5,6], [7,8]],
              'names::string':   ['john', 'eric', 'judith', 'max'],
              'index':           [1, 2, 3, 4],
              'complete_test':   [['a', 'b'], [0, 0, 1, 0]],
              'complete_date':   [{"::date": ["2000-01-01", "2000-02-01"]}, [0, 0, 1, 0]],
              'implicit_test':   [['a', 'b'], [1]],
              'implicit_test2':  [['a', 'b'], [2]],
              'unic_test':       'valunic' }
il = from_ntv_ilist(tab_data)
'''for idx in il.lindex:
    print('idx : ', idx.name, idx.codec, idx.keys)
for info in il.indexinfos():
    print('indexinfos : ', info)
print('\nil : ', il.lname, il.idxname, il.lvarname, il.primary) #, il.primaryname)
ntv = Ntv.obj(tab_data)
#print(ntv)
lidx = [list(decode_ntv_field(ntvf)) for ntvf in ntv]
print('\ntest', lidx, '\n\n')'''
for mode in ['full', 'default', 'optimize']:
    #print(to_ntv_ilist(il, mode))
    il2 = from_ntv_ilist(to_ntv_ilist(il, mode))
    print(mode, il == il2)
    #il2.analysis.actualize()
    if il != il2:
        print(to_ntv_ilist(il, mode))
        print(il.lvarname)
        print(il2.lvarname)

full True
default True
optimize False
{"Ilist": [{"value": [10, 20, 30, 40]}, {"dates::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00", "2022-01-22T00:00:00+00:00"]}, {"value32": [10, 20, 30, 40]}, {"coord": [[1, 2], [3, 4], [5, 6], [7, 8]]}, {"names": ["john", "eric", "judith", "max"]}, {"index": [1, 2, 3, 4]}, {"complete_test": [["a", "b"], [0, 0, 1, 0]]}, {"complete_date": [{"::datetime": ["2000-01-01T00:00:00+00:00", "2000-02-01T00:00:00+00:00"]}, 6, [0, 0, 1, 0]]}, {"implicit_test": [["a", "b"], [1]]}, {"implicit_test2": [["a", "b"], [2]]}, {"unic_test": "valunic"}]}
['value', 'complete_test', 'dates', 'value32', 'coord', 'names', 'index', 'complete_date']
[]


In [20]:
for idx in il.lindex:
    print('idx : ', idx.name, idx.codec, idx.keys)
for info in il.indexinfos():
    print('indexinfos : ', info)

idx :  value [10, 20, 30, 40] [0, 1, 2, 3]
idx :  dates [datetime.datetime(1964, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(1985, 2, 5, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 1, 21, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 1, 22, 0, 0, tzinfo=datetime.timezone.utc)] [0, 1, 2, 3]
idx :  value32 [10, 20, 30, 40] [0, 1, 2, 3]
idx :  coord [(1, 2), (3, 4), (5, 6), (7, 8)] [0, 1, 2, 3]
idx :  names ['john', 'eric', 'judith', 'max'] [0, 1, 2, 3]
idx :  index [1, 2, 3, 4] [0, 1, 2, 3]
idx :  complete_test ['a', 'b'] [0, 0, 1, 0]
idx :  complete_date [datetime.datetime(2000, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2000, 2, 1, 0, 0, tzinfo=datetime.timezone.utc)] [0, 0, 1, 0]
idx :  implicit_test ['a', 'b'] [0, 1, 0, 1]
idx :  implicit_test2 ['a', 'b'] [0, 0, 1, 1]
idx :  unic_test ['valunic'] [0, 0, 0, 0]
indexinfos :  {'num': 0, 'name': 'value', 'cat': 'variable', 'parent': -1, 'distparent': -1, 'minparent': -1, 

In [19]:
for idx in il2.lindex:
    print('idx : ', idx.name, idx.codec, idx.keys)
for info in il2.indexinfos():
    print('indexinfos : ', info)

idx :  value [10, 20, 30, 40] [0, 1, 2, 3]
idx :  dates [datetime.datetime(1964, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(1985, 2, 5, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 1, 21, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 1, 22, 0, 0, tzinfo=datetime.timezone.utc)] [0, 1, 2, 3]
idx :  value32 [10, 20, 30, 40] [0, 1, 2, 3]
idx :  coord [(1, 2), (3, 4), (5, 6), (7, 8)] [0, 1, 2, 3]
idx :  names ['john', 'eric', 'judith', 'max'] [0, 1, 2, 3]
idx :  index [1, 2, 3, 4] [0, 1, 2, 3]
idx :  complete_test ['a', 'b'] [0, 0, 1, 0]
idx :  complete_date [datetime.datetime(2000, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2000, 2, 1, 0, 0, tzinfo=datetime.timezone.utc)] [0, 0, 0, 0]
idx :  implicit_test ['a', 'b'] [0, 1, 0, 1]
idx :  implicit_test2 ['a', 'b'] [0, 0, 1, 1]
idx :  unic_test ['valunic'] [0, 0, 0, 0]
indexinfos :  {'num': 0, 'name': 'value', 'cat': 'primary', 'parent': -1, 'distparent': -1, 'minparent': -1, '

In [10]:
tab_data   = [['1964-01-01', '1985-02-05', '2022-01-21', '2022-01-22'], 
              [10, 20, 30, 40],
              [10, 20, 30, 40],
              [[1,2], [3,4], [5,6], [7,8]],
              ['john', 'eric', 'judith', 'max'],
              [1, 2, 3, 4],
              [['a', 'b'], [0, 0, 1, 0]],
              [['a', 'b'], [1]],
              [['a', 'b'], [2]],
              'valunic' ]

print(from_ntv_ilist(tab_data))
for mode in ['full', 'default', 'optimize']:
    print(to_ntv_ilist(il, mode))
    print(il == from_ntv_ilist(to_ntv_ilist(il, mode)))

    ["i0", ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00", "2022-01-22T00:00:00+00:00"]]
    ["i6", ["a", "a", "b", "a"]]
    ["i1", [10, 20, 30, 40]]
    ["i2", [10, 20, 30, 40]]
    ["i3", [[1, 2], [3, 4], [5, 6], [7, 8]]]
    ["i4", ["john", "eric", "judith", "max"]]
    ["i5", [1, 2, 3, 4]]

    ["i7", ["a", "b", "a", "b"]]
    ["i8", ["a", "a", "b", "b"]]
    ["i9", ["valunic", "valunic", "valunic", "valunic"]]

{"Ilist": [{"value": [10, 20, 30, 40]}, {"dates::datetime": ["1964-01-01T00:00:00+00:00", "1985-02-05T00:00:00+00:00", "2022-01-21T00:00:00+00:00", "2022-01-22T00:00:00+00:00"]}, {"value32": [10, 20, 30, 40]}, {"coord": [[1, 2], [3, 4], [5, 6], [7, 8]]}, {"names": ["john", "eric", "judith", "max"]}, {"index": [1, 2, 3, 4]}, {"complete_test": ["a", "a", "b", "a"]}, {"complete_date::datetime": ["2000-01-01T00:00:00+00:00", "2000-01-01T00:00:00+00:00", "2000-02-01T00:00:00+00:00", "2000-01-01T00:00:00+00:00"]}, {"implicit_test": ["a", "

In [11]:

tab_data   = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, 30],
              'value32::int32':  [10, 20, 30],
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith'],
              'index':           [1, 2, 3]}

field = Ntv.obj({':field': field_data})
fiel2 = Ntv.obj({':field': field_dat2})
tab   = Ntv.obj({':tab'   : tab_data})

# the DataFrame Connector is associated with NtvType 'tab' in dicobj 
sr = fiel2.to_obj(encode_format='obj', dicobj={'field': 'SeriesConnec'})
df = tab.to_obj  (encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

# pandas dtype conform to Ntv type
print(sr, '\n')
print(df.dtypes)
df

NameError: name 'field_data' is not defined

In [ ]:
import json
type_to_dtype = {'date': 'datetime64[ns]', 'datetime': 'datetime64[ns]', 'string': 'string', 'int32': 'int32', 'int64': 'int64' }
dtype_to_type = {'datetime64[ns]': 'datetime', 'string': 'string', 'int32': 'int32', 'int64': 'json'}
#print(dtype_to_type)

def to_ntv_sr(sr):
    ntv_type = None
    dtype = sr.dtype.name
    if dtype in dtype_to_type:
        if dtype_to_type[dtype] != 'json':
            ntv_type = dtype_to_type[dtype]
        ntv_value = json.loads(sr.to_json(orient='records', date_format='iso', default_handler=str))
    elif sr.dtype.name == 'object':
        ntv_value = sr.to_list()
    ntv_name = sr.name
    return NtvSingle(NtvList(ntv_value, ntv_name, ntv_type).to_obj(), None, 'field')
    
def to_ntv_df(df):
    df2 = df.reset_index()
    return NtvSingle(NtvSet([to_ntv_sr(df2[colon]).ntv_value for colon in df2.columns]).to_obj(), None, 'tab')
    #print([df[colon].to_list() for colon in df.columns])

In [ ]:
print(to_ntv_sr(df.value).to_obj(), '\n')
print(to_ntv_df(df))

In [ ]:
sr = from_ntv_sr(tab[0])
sr2 = from_ntv_sr(tab[3])
df = from_ntv_df(tab)
print(sr.name, sr.dtype, type(sr.dtype), sr2.name, sr2.dtype)
print(df.dtypes)
print(to_ntv_sr(df['dates']))
print(to_ntv_df(df))

In [ ]:
lis = [df[]]
print(df)

In [ ]:
print(Series(['john', 'eric', 'judith'], dtype='string'))
print(tab[3])
print(tab[3].to_obj(encode_format='obj', simpleval=True))